In [ ]:
# default_exp token_classification

# Token Tagging and Classification
> AdaptiveModels for using Transformers and Flair for token tagging and classification

In [ ]:
#hide
from fastcore.test import test_eq
from nbdev.showdoc import show_doc

In [ ]:
#export
import logging
from typing import List, Dict, Union
from collections import defaultdict

import numpy as np

import torch
from torch.utils.data import TensorDataset

from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    PreTrainedTokenizer,
    PreTrainedModel,
    BertForSequenceClassification,
    XLNetForSequenceClassification,
    AlbertForSequenceClassification,
)

from adaptnlp.model import AdaptiveModel, DataLoader
from adaptnlp.model_hub import HFModelResult, FlairModelResult, FlairModelHub, HFModelHub

from fastai_minima.utils import to_detach, apply, to_device

from fastcore.basics import Self, risinstance
from fastcore.xtras import Path

In [ ]:
#export
logger = logging.getLogger(__name__)

In [ ]:
#export
class TransformersTokenTagger(AdaptiveModel):
    """Adaptive model for Transformer's Token Tagger Model

    Usage:
    ```python
    >>> tagger = TransformersTokenTagger.load("dbmdz/bert-large-cased-finetuned-conll03-english")
    >>> tagger.predict(text="Example text", mini_batch_size=32)
    ```

    **Parameters:**

    * **tokenizer** - A tokenizer object from Huggingface's transformers (TODO) and tokenizers
    * **model** - A transformers token tagger model
    """

    def __init__(self, tokenizer: PreTrainedTokenizer, model: PreTrainedModel):
        # Load up model and tokenizer
        self.tokenizer = tokenizer
        super().__init__()

        # Sets the internal model
        self.set_model(model)

    @classmethod
    def load(cls, model_name_or_path: str) -> AdaptiveModel:
        """Class method for loading and constructing this tagger

        * **model_name_or_path** - A key string of one of Transformer's pre-trained Token Tagger Model or a `HFModelResult`
        
        Note: To search for valid models, you should use the AdaptNLP `model_hub` API
        """
        if isinstance(model_name_or_path, HFModelResult): model_name_or_path = model_name_or_path.name
        tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        model = AutoModelForTokenClassification.from_pretrained(model_name_or_path)
        tagger = cls(tokenizer, model)
        return tagger

    def predict(
        self,
        text: Union[List[str], str],
        mini_batch_size: int = 32,
        grouped_entities: bool = True,
        **kwargs,
    ) -> List[List[Dict]]:
        """Predict method for running inference using the pre-trained token tagger model.
        Returns a list of lists of tagged entities.

        * **text** - String, list of strings, sentences, or list of sentences to run inference on
        * **mini_batch_size** - Mini batch size
        * **grouped_entities** - Set True to get whole entity span strings (Default True)
        * **&ast;&ast;kwargs**(Optional) - Optional arguments for the Transformers tagger
        """
        if isinstance(text, str):
            text = [text]
        results: List[Dict] = []

        dataset = self._tokenize(text)
        dl = DataLoader(dataset, batch_size=mini_batch_size)

        logger.info(f'Running prediction on {len(dataset)} text sequences')
        logger.info(f'Batch size = {mini_batch_size}')

        outputs,_ = super().get_preds(dl=dl)

        inputs = apply(to_device, [b for b in dl], device='cpu')
        inputs = torch.cat(*inputs)
        inputs = apply(Self.numpy(), inputs)

        outputs = torch.cat([o['logits'] for o in outputs])
        outputs = apply(to_detach, outputs, cpu=True)
        outputs = apply(Self.numpy(), outputs)

        # Iterate through batch for tagged token predictions
        for idx, pred in enumerate(outputs):
            entities = pred
            input_ids = inputs[idx]
            tagged_entities = self._generate_tagged_entities(
                entities=entities,
                input_ids=input_ids,
                grouped_entities=grouped_entities
            )
            results += tagged_entities

        return results

    def _tokenize(
        self, sentences: Union[List[Sentence], Sentence, List[str], str]
    ) -> TensorDataset:
        """ Batch tokenizes text and produces a `TensorDataset` with them """

        tokenized_text = self.tokenizer.batch_encode_plus(
            sentences,
            return_tensors="pt",
            pad_to_max_length=True,
        )

        # Bart, XLM, DistilBERT, RoBERTa, and XLM-RoBERTa don't use token_type_ids
        if isinstance(
            self.model,
            (
                BertForSequenceClassification,
                XLNetForSequenceClassification,
                AlbertForSequenceClassification,
            ),
        ):
            dataset = TensorDataset(
                tokenized_text["input_ids"],
                tokenized_text["attention_mask"],
                tokenized_text["token_type_ids"],
            )
        else:
            dataset = TensorDataset(
                tokenized_text["input_ids"], tokenized_text["attention_mask"]
            )

        return dataset

    # `_group_entites` and `_generate_tagged_entities` modified from pipeline code snippet from Transformers
    def _group_entities(
        self, entities: List[dict], idx_start: int, idx_end: int
    ) -> Dict:
        """Returns grouped entities"""
        # Get the last entity in the entity group
        entity = entities[-1]["entity"]
        scores = np.mean([entity["score"] for entity in entities])
        tokens = [entity["word"] for entity in entities]

        entity_group = {
            "entity_group": entity,
            "score": np.mean(scores),
            "word": self.tokenizer.convert_tokens_to_string(tokens),
            "offsets": (idx_start, idx_end),
        }
        return entity_group

    def _generate_tagged_entities(
        self, entities: np.ndarray, input_ids: np.ndarray, grouped_entities: bool = True
    ) -> List[Dict]:
        """Generate full list of entities given tagged token predictions and input_ids"""

        score = np.exp(entities) / np.exp(entities).sum(-1, keepdims=True)
        labels_idx = score.argmax(axis=-1)

        answers = []
        entities = []
        entity_groups = []
        entity_group_disagg = []
        # Filter to labels not in `self.ignore_labels`
        filtered_labels_idx = [
            (idx, label_idx)
            for idx, label_idx in enumerate(labels_idx)
            if self.model.config.id2label[label_idx] not in ["O"]
        ]

        for idx, label_idx in filtered_labels_idx:
            # print(tokenizer.convert_ids_to_tokens(int(input_ids[idx])))
            entity = {
                "word": self.tokenizer.convert_ids_to_tokens(int(input_ids[idx])),
                "score": score[idx][label_idx].item(),
                "entity": self.model.config.id2label[label_idx],
                "index": idx,
            }
            last_idx, _ = filtered_labels_idx[-1]
            if grouped_entities:
                if not entity_group_disagg:
                    entity_group_disagg += [entity]
                    if idx == last_idx:
                        entity_groups += [
                            self._group_entities(
                                entity_group_disagg, idx - len(entity_group_disagg), idx
                            )
                        ]
                    continue

                # If the current entity is similar and adjacent to the previous entity, append it to the disaggregated entity group
                if (
                    entity["entity"] == entity_group_disagg[-1]["entity"]
                    and entity["index"] == entity_group_disagg[-1]["index"] + 1
                ):
                    entity_group_disagg += [entity]
                    # Group the entities at the last entity
                    if idx == last_idx:
                        entity_groups += [
                            self._group_entities(
                                entity_group_disagg, idx - len(entity_group_disagg), idx
                            )
                        ]
                # If the current entity is different from the previous entity, aggregate the disaggregated entity group
                else:
                    entity_groups += [
                        self._group_entities(
                            entity_group_disagg,
                            entity_group_disagg[-1]["index"] - len(entity_group_disagg),
                            entity_group_disagg[-1]["index"],
                        )
                    ]
                    entity_group_disagg = [entity]

            entities += [entity]

        # Append
        if grouped_entities:
            answers += [entity_groups]
        else:
            answers += [entities]

        return answers

    def train(
        self,
    ):
        raise NotImplementedError

    def evaluate(
        self,
    ):
        raise NotImplementedError

In [ ]:
#hide
tagger = TransformersTokenTagger.load("dbmdz/bert-large-cased-finetuned-conll03-english")
pred = tagger.predict(text='Novetta Solutions is the best. Albert Einstein used to be employed at Novetta Solutions. The Wright brothers loved to visit the JBF headquarters, and they would have a chat with Albert.', mini_batch_size=32)
baseline = [[{'entity_group': 'I-ORG',
   'score': 0.998292068640391,
   'word': 'Novetta Solutions',
   'offsets': (0, 3)},
  {'entity_group': 'I-PER',
   'score': 0.9985582232475281,
   'word': 'Albert Einstein',
   'offsets': (7, 9)},
  {'entity_group': 'I-ORG',
   'score': 0.9970489343007406,
   'word': 'Novetta Solutions',
   'offsets': (14, 17)},
  {'entity_group': 'I-PER',
   'score': 0.9961656928062439,
   'word': 'Wright',
   'offsets': (19, 20)},
  {'entity_group': 'I-ORG',
   'score': 0.9933501183986664,
   'word': 'JBF',
   'offsets': (25, 27)}]]

for base, p in zip(baseline, pred):
    test_eq(base['entity_group'], p['entity_group'])
    test_close(base['score'], p['score'], 1e-3)
    test_eq(base['word'], p['word'])
    test_eq(base['offsets'], p['offsets'])

/opt/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2073: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
#export
class FlairTokenTagger(AdaptiveModel):
    """Adaptive Model for Flair's Token Tagger...very basic

    Usage:
    ```python
    >>> tagger = FlairTokenTagger.load("flair/chunk-english-fast")
    >>> tagger.predict(text="Example text", mini_batch_size=32)
    ```

    **Parameters:**

    * **model_name_or_path** - A key string of one of Flair's pre-trained Token tagger Model
    
    To find a list of available models, see [here](https://huggingface.co/models?filter=flair)
    """

    def __init__(self, model_name_or_path: str):
        self.tagger = SequenceTagger.load(model_name_or_path)

    @classmethod
    def load(cls, model_name_or_path: str) -> AdaptiveModel:
        """Class method for loading a constructing this tagger

        * **model_name_or_path** - A key string of one of Flair's pre-trained Token tagger Model
        
        To find a list of available models, see [here](https://huggingface.co/models?filter=flair)
        """
        tagger = cls(model_name_or_path)
        return tagger

    def predict(
        self,
        text: Union[List[Sentence], Sentence, List[str], str],
        mini_batch_size: int = 32,
        **kwargs,
    ) -> List[Sentence]:
        """Predict method for running inference using the pre-trained token tagger model

        * **text** - String, list of strings, sentences, or list of sentences to run inference on
        * **mini_batch_size** - Mini batch size
        * **&ast;&ast;kwargs**(Optional) - Optional arguments for the Flair tagger
        """

        if isinstance(text, (Sentence, str)):
            text = [text]
        if isinstance(text[0], str):
            text = [Sentence(s) for s in text]
        self.tagger.predict(
            sentences=text,
            mini_batch_size=mini_batch_size,
            **kwargs,
        )
        return text

    def train(
        self,
    ):
        raise NotImplementedError

    def evaluate(
        self,
    ):

        raise NotImplementedError

In [ ]:
#hide
tagger = FlairTokenTagger.load("flair/chunk-english-fast")
preds = tagger.predict(text="Example text", mini_batch_size=32)[0]
test_eq(preds.tokens[0].text, 'Example')
test_eq(preds.tokens[1].text, 'text')

2021-04-20 18:27:34,432 loading file /root/.flair/models/chunk-english-fast/be3a207f4993dd6d174d5083341a717d371ec16f721358e7a4d72158ebab28a6.a7f897d05c83e618a8235bbb7ddfca5a79d2daefb8a97c776eb73f97dbaea508


In [ ]:
#export
class EasyTokenTagger:
    """Token level classification models

    Usage:

    ```python
    >>> tagger = adaptnlp.EasyTokenTagger()
    >>> tagger.tag_text(text="text you want to tag", model_name_or_path="ner-ontonotes")
    ```
    """

    def __init__(self):
        self.token_taggers: Dict[AdaptiveModel] = defaultdict(bool)

    def tag_text(
        self,
        text: Union[List[Sentence], Sentence, List[str], str],
        model_name_or_path: Union[str, FlairModelResult, HFModelResult] = "ner-ontonotes",
        mini_batch_size: int = 32,
        **kwargs,
    ) -> List[Sentence]:
        """Tags tokens with labels the token classification models have been trained on

        * **text** - Text input, it can be a string or any of Flair's `Sentence` input formats
        * **model_name_or_path** - The hosted model name key or model path
        * **mini_batch_size** - The mini batch size for running inference
        * **&ast;&ast;kwargs** - Keyword arguments for Flair's `SequenceTagger.predict()` method
        **return** - A list of Flair's `Sentence`'s
        """
        # Load Sequence Tagger Model and Pytorch Module into tagger dict
        name = getattr(model_name_or_path, 'name', model_name_or_path)
        if not self.token_taggers[name]:
            """
            self.token_taggers[model_name_or_path] = SequenceTagger.load(
                model_name_or_path
            )
            """
            if risinstance([FlairModelResult, HFModelResult], model_name_or_path):
                try:
                    self.token_taggers[name] = FlairTokenTagger.load(name)
                except:
                    self.token_taggers[name] = TransformersTokenTagger.load(name)
            elif risinstance([str, Path], model_name_or_path) and (Path(model_name_or_path).exists() and Path(model_name_or_path).is_dir()):
                # Load in previously existing model
                try:
                    self.token_taggers[name] = FlairTokenTagger.load(name)
                except:
                    self.token_taggers[name] = TransformersTokenTagger.load(name)
            else:
                _flair_hub = FlairModelHub()
                _hf_hub = HFModelHub()
                res = _flair_hub.search_model_by_name(name, user_uploaded=True)
                if len(res) < 1:
                    # No models found
                    res = _hf_hub.search_model_by_name(name, user_uploaded=True)
                    if len(res) < 1:
                        logger.info("Not a valid `model_name_or_path` param")
                        return [Sentence('')]
                    else:
                        res[0].name.replace('flairNLP', 'flair')
                        self.token_taggers[res[0].name] = TransformersTokenTagger.load(res[0].name)
                        name = res[0].name

                else:
                    name = res[0].name.replace('flairNLP/', '')
                    self.token_taggers[name] = FlairTokenTagger.load(name) # Returning the first should always be the non-fast option
                    
        tagger = self.token_taggers[name]
        return tagger.predict(
            text=text,
            mini_batch_size=mini_batch_size,
            **kwargs,
        )

    def tag_all(
        self,
        text: Union[List[Sentence], Sentence, List[str], str],
        mini_batch_size: int = 32,
        **kwargs,
    ) -> List[Sentence]:
        """Tags tokens with all labels from all token classification models

        * **text** - Text input, it can be a string or any of Flair's `Sentence` input formats
        * **mini_batch_size** - The mini batch size for running inference
        * **&ast;&ast;kwargs** - Keyword arguments for Flair's `SequenceTagger.predict()` method
        **return** A list of Flair's `Sentence`'s
        """
        if len(self.token_taggers) == 0:
            print("No token classification models loaded...")
            return Sentence()
        sentences = text
        for tagger_name in self.token_taggers.keys():
            sentences = self.tag_text(
                sentences,
                model_name_or_path=tagger_name,
                mini_batch_size=mini_batch_size,
                **kwargs,
            )
        return sentences

In [ ]:
#hide
tagger = EasyTokenTagger()
example_text = '''Novetta Solutions is the best. Albert Einstein used to be employed at Novetta Solutions. 
The Wright brothers loved to visit the JBF headquarters, and they would have a chat with Albert.'''
sentences = ["Jack walked through the park on a Sunday.", "Sunday was a nice and breezy afternoon.", "Jack was going to meet Jill for dinner."]
tagger.tag_text(text=example_text, model_name_or_path="ner-ontonotes")
tagger.tag_text(text=example_text, model_name_or_path="pos")
tags = tagger.tag_all(sentences)
_types = [["PERSON", "DATE"], ["DATE"], ["PERSON", "PERSON"]]

for sentence, lbls in zip(tags, _types):
    spans = sentence.get_spans()
    for span, lbl in zip(spans, lbls):
        test_eq(span.tag, lbl)

2021-04-20 18:30:25,272 --------------------------------------------------------------------------------
2021-04-20 18:30:25,273 The model key 'ner-ontonotes' now maps to 'https://huggingface.co/flair/ner-english-ontonotes' on the HuggingFace ModelHub
2021-04-20 18:30:25,273  - The most current version of the model is automatically downloaded from there.
2021-04-20 18:30:25,274  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner-ontonotes/en-ner-ontonotes-v0.4.pt)
2021-04-20 18:30:25,275 --------------------------------------------------------------------------------
2021-04-20 18:30:25,295 loading file /root/.flair/models/ner-english-ontonotes/f46dcd14689a594a7dd2a8c9c001a34fd55b02fded2528410913c7e88dbe43d4.1207747bf5ae24291205b6f3e7417c8bedd5c32cacfb5a439f3eff38afda66f7
2021-04-20 18:30:31,136 loading file /root/.flair/models/pos-english-fast/36f7923039eed4c66e4275927daaff6cd275997d61d238355fb1fe0338fe10a1.ff87e5b